## An Alternate Method for Data Acquisition

I noticed some issues with the previous dataset. Notably, the volume column contained many zero values. With enough incorrect values from the source data, errors can propogate to future pipeline steps. To correct this, I will instead use [The Investors Exchange](https://iextrading.com/), which provides free historical stock data. This data is more accurate and reliable than the Kaggle repo sourced from Yahoo Finance.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

To acquire the data, an API key from [Alpaca](https://alpaca.markets/) is required. Alpaca provides an easy to use API for stock trading, market data acquisition, and backtesting. Some features require an authorized account, but for my needs a basic account will work. To utilize the API  you must first generate two keys, a key ID and a secret key. 

In [2]:
import alpaca_trade_api as tradeapi

key_id = None
secret_key = None
with open('../files/private/alpakey') as key_file:
    keys = key_file.readlines()
    key_id = keys[0].strip()
    secret_key = keys[1].strip()
    
api_url = "https://paper-api.alpaca.markets"

alpaca = tradeapi.REST(key_id, secret_key, api_url, api_version='v2')

'top_symbols.json' contains a list of the 500 most frequently traded stock symbols, compiled by scraping The Investors Exchange's website.

In [3]:
import json

symbols = []
with open('../files/public/top_symbols.json') as top_symbols:
    symbols = json.load(top_symbols)

The API restricts users to 100 symbols per call. You can specify the timeperiod between each data point, the number of data points, and the starting or ending point. The results can be returned as a DataFrame for convenience. To train the model, the last 1000 days of data will be used. This data dates back to the middle of 2016.

In [ ]:
top_stocks = []
for symbol in tqdm(symbols):
    try:
        stock = alpaca.polygon.historic_agg_v2(symbol, 1, 'day', _from='2008-01-01', to='2020-05-01').df
        stock['symbol'] = symbol
        top_stocks.append(stock)
    except:
        continue
        
top_stocks = pd.concat(top_stocks)

In [ ]:
top_stocks.reset_index(inplace=True)
top_stocks.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'symbol']

For backtesting the model, unseen data will need to be acquired. However, due to fluctuations in the market, not every year makes for balanced data. To avoid this, I will backtest using three sets of data. 

##### Backtest Set 1: 2008
    
The market crash of 2008 was one of the worst on record. I'll choose this year to test the model's performance in the face of a recession.
    
##### Backtest Set 2: 2011

2011 was neither a good year or a bad year for the market. There was little difference between stock prices in January and December. I'll choose this year to test the model's performance in flat years.

##### Backtest Set 3: 2013

By 2013, the market had rebounded from the 2008 recession. Throughout the year, stock growth never stopped. I'll choose this year to test the model's performance in a successful market.

In [ ]:
top_stocks.set_index('date', inplace=True)

stocks_current = top_stocks['2015':'2020-05-14']
stocks_2008 = top_stocks['2007':'2008']
stocks_2011 = top_stocks['2010':'2011']
stocks_2013 = top_stocks['2012':'2013']

After the stocks have been retrieved, I concatenate them into a single DataFrame.

Some ETFs (Exchange Traded Funds) track multiple stocks and can be used to approximate the market as a whole. The SPY ETF tracks the S&P 500, a collection of stocks listed on the US markets. By combining stock data with overall market data, better predictions can be made that take into account market ups and downs. Other ETFs may work just as well, such as DIA (Dow Jones Industrial Average) or VUG (Vanguard). In my tests, they all performed similarly.

In [ ]:
market = alpaca.polygon.historic_agg_v2('SPY', 1, 'day', _from='2007-01-01', to='2020-05-14').df
market.reset_index(inplace=True)
market.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

In [ ]:
stocks_current = stocks_current.reset_index().merge(market, on='date', suffixes=['', '_market'])
stocks_2008 = stocks_2008.reset_index().merge(market, on='date', suffixes=['', '_market'])
stocks_2011 = stocks_2011.reset_index().merge(market, on='date', suffixes=['', '_market'])
stocks_2013 = stocks_2013.reset_index().merge(market, on='date', suffixes=['', '_market'])

The stock DataFrames are merged with the market DataFrame by date.

In [ ]:
stocks_current.set_index('date', inplace=True)
stocks_2008.set_index('date', inplace=True)
stocks_2011.set_index('date', inplace=True)
stocks_2013.set_index('date', inplace=True)

stocks_current.drop_duplicates(inplace=True)
stocks_2008.drop_duplicates(inplace=True)
stocks_2011.drop_duplicates(inplace=True)
stocks_2013.drop_duplicates(inplace=True)

The finalized data is saved for future use. This step may be skipped in future iterations once you are satisfied with the size and quality of the dataset.

In [ ]:
stocks_current.to_hdf('../data/raw/market_stocks.h5', key='stocks_current')
stocks_2008.to_hdf('../data/raw/market_stocks.h5', key='stocks_2008')
stocks_2011.to_hdf('../data/raw/market_stocks.h5', key='stocks_2011')
stocks_2013.to_hdf('../data/raw/market_stocks.h5', key='stocks_2013')